<a href="https://colab.research.google.com/github/Malekkamoua/Medical-data-classifier/blob/main/4_Summarize_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
 
drive.mount('/content/gdrive')

In [ ]:
!unzip  "/content/gdrive/MyDrive/PFA DATA/Clean_data/Copy of clean_data.zip"

In [ ]:
import os 
from shutil import copyfile

path = "/content/content/supervised_sample"
os.mkdir(path)

print("created "+ path)

count = 0
for file in os.listdir(f"/content/content/clean_data"):
  if count < 20000:
    copyfile(f"/content/content/clean_data/{file}",f"{path}/{file}")
    count  = count + 1

print(count)
print("Done")

In [ ]:
pip install nltk==3.4.5

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
!pip install sumy

In [ ]:
import re
import nltk

def clean_text(x):
    x = " ".join(x.split())
    x= " ".join((" ".join(x.split("[**"))).split("**]"))
    x = re.sub(r"\([^()]*\)", "", x)
    key_value_strip =(x.split(":"))
    ##remove all sub strings which have a length lesser than 50 characters
    string = " ".join([sub_unit for sub_unit in key_value_strip if len(sub_unit)>50])
    x = re.sub(r"(\d+)+(\.|\))", "", string)## remove all serialization eg 1. 1)
    x = re.sub(r"(\*|\?|=)+", "", x) ##removing all *, ? and =
    x = re.sub(r"\b(\w+)( \1\b)+", r"\1", x) ## removing consecutive dupicate words
    x = x.replace("FOLLOW UP", "FOLLOWUP")
    x = x.replace("FOLLOW-UP", "FOLLOWUP")
    x = re.sub(r"(\b)(f|F)(irst)(\b)?[\d\-\d]*(\s)*(\b)?(n|N)(ame)[\d\-\d]*(\s)*[\d\-\d]*(\b)","",x)##remove firstname
    x = re.sub(r"(\b)(l|L)(ast)(\b)?[\d\-\d]*(\s)*(\b)?(n|N)(ame)[\d\-\d]*(\s)*[\d\-\d]*(\b)", "", x)
    x = re.sub(r"(\b)(d|D)\.?(r|R)\.?(\b)", "", x) #remove Dr abreviation
    x = re.sub(r"([^A-Za-z0-9\s](\s)){2,}", "", x)##remove consecutive punctuations
    x = re.sub(r'\d+', '', x)   #Remove numbers
    x = x.lower() #To Lower case
    return(x.replace("  ", " "))

def ignore_non_ascii(text):
  text = text.encode("ascii", "ignore")
  string_decode = text.decode()

  return string_decode

def only_english(text):

  words = set(nltk.corpus.words.words())
  clean = " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha())

  return clean

def only_period_punct (text):

  punctuations = '''!()-[]{};:'"\,<>|°/?@#$%^&*_~©'''
  no_punc = ""
  for char in text:
    if char not in punctuations:
        no_punc = no_punc + char
  
  return no_punc

def remove_long_words(text):
  #The average word length in English language is 4.7 characters.
  # remove words between 10 and 100
  shortword = re.compile(r'\W*\b\w{10,100}\b')
  return shortword.sub('', text)

def remove_short_words(text):
  #The average word length in English language is 4.7 characters.
  # remove words between 1 and 2
  shortword = re.compile(r'\W*\b\w{1,2}\b')
  return shortword.sub('', text)

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

# Import the LexRank summarizer
from sumy.summarizers.lex_rank import LexRankSummarizer

def summarize(text):

  # Initializing the parser
  my_parser = PlaintextParser.from_string(text,Tokenizer('english'))
  # Creating a summary of 3 sentences.
  lex_rank_summarizer = LexRankSummarizer()
  lexrank_summary = lex_rank_summarizer(my_parser.document,sentences_count=3)

  summary = ""
  # Printing the summary
  for sentence in lexrank_summary:
    summary = summary + str(sentence)
  
  return summary
    

In [ ]:
import re
import string
global str
import nltk  
from nltk.tokenize import word_tokenize 

In [ ]:
# iterate through all files
doc = []
for file in os.listdir(path):
    text = " "
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"
        with open(file_path, 'r') as f:
          text = f.readline()

          #preprocessing
          res_clean_text = clean_text(text)
          res_ignore_non_ascii = ignore_non_ascii(res_clean_text)
          res_only_period_punct = only_period_punct(res_ignore_non_ascii)
          res_only_english = only_english(res_only_period_punct)
          res_remove_long_words = remove_long_words(res_only_english)
          res_remove_short_words = remove_short_words(res_remove_long_words)

          if res_remove_short_words !=  "":
            doc.append(res_remove_short_words)

In [ ]:
len(doc)

15545

In [ ]:
from nltk.corpus import stopwords

tmp = []
summarized_data = []

for file in doc:
  #Removing stop words
  stop_words = set(stopwords.words('english')) 
  word_tokens = word_tokenize(file)       
  content = [w for w in word_tokens if not w in stop_words] 
  filtered_sentence = (" ").join(content)
  tmp.append(filtered_sentence)

for file in tmp:
  new_file = summarize(file)
  summarized_data.append(new_file)

In [ ]:
len(summarized_data)

15545